## Deduplicating data

In this notebook, we deduplicate data using the [Dedupe library](https://dedupe.readthedocs.io/en/latest/), which utilizes a shallow neural network to learn from a small training exercise.

If you are interested in building your own parser, the same folks have created the [Parserator](https://github.com/datamade/parserator) which you can use to extract text features and train your own text extraction (hooray! less brittle than regex!)

In [2]:
import pandas as pd
import dedupe
import os

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [3]:
customers = pd.read_csv('../data/customer_data_duped.csv', 
                        encoding='utf-8')

## Checking Data Quality

In [4]:
customers.head()

,name,job,company,street_address,city,state,email,user_name
0,Patricia Schaefer,"Programmer, systems",Estrada-Best,398 Paul Drive,Christianview,Delaware,lambdavid@gmail.com,ndavidson
1,Olivie Dubois,Ingénieur recherche et développement en agroal...,Moreno,rue Lucas Benard,Saint Anastasie-les-Bains,AR,berthelotjacqueline@mahe.fr,manonallain
2,Mary Davies-Kirk,Public affairs consultant,Baker Ltd,Flat 3\nPugh mews,Stanleyfurt,ZA,middletonconor@hotmail.com,colemanmichael
3,Miroslawa Eckbauer,Dispensing optician,Ladeck GmbH,Mijo-Lübs-Straße 12,Neubrandenburg,Berlin,sophia01@yahoo.de,romanjunitz
4,Richard Bauer,"Accountant, chartered certified",Hoffman-Rocha,6541 Rodriguez Wall,Carlosmouth,Texas,tross@jensen-ware.org,adam78


In [5]:
customers.dtypes

name              object
job               object
company           object
street_address    object
city              object
state             object
email             object
user_name         object
dtype: object

In [6]:
for col in customers.columns:
    print(col, customers[col].isnull().sum())

name 0
job 0
company 0
street_address 0
city 0
state 0
email 0
user_name 0


## Setting up Dedupe

In [7]:
variables = [
    {'field': 'name', 'type': 'String'},
    {'field': 'job', 'type': 'String'},
    {'field': 'company', 'type': 'String'},  
    {'field': 'street_address','type': 'String'},
    {'field': 'city','type': 'String'},
    {'field': 'state', 'type': 'String', 'has_missing': True},
    {'field': 'email', 'type': 'String', 'has_missing': True},
    {'field': 'user_name', 'type': 'String'},
]

deduper = dedupe.Dedupe(variables)

In [8]:
deduper

In [9]:
customers.shape

(2080, 8)

In [13]:
customers.T.to_dict()[0]

{'city': 'Christianview',
 'company': 'Estrada-Best',
 'email': 'lambdavid@gmail.com',
 'job': 'Programmer, systems',
 'name': 'Patricia Schaefer',
 'state': 'Delaware',
 'street_address': '398 Paul Drive',
 'user_name': 'ndavidson'}

In [15]:
deduper.sample(customers.T.to_dict(), 249)

Note: If you receive an error like this:

```/usr/local/lib/python2.7/site-packages/dedupe/sampling.py:39: UserWarning: 250 blocked samples were requested, but only able to sample 249
  % (sample_size, len(blocked_sample)))
```

you can continue (some were selected), or use the suggested number (^ here it would be 249)

#### Either use training file (uncomment) or resume active training below

In [16]:
training_file = '../data/ignore-dedupe-training.json'
#if os.path.exists(training_file):
#    with open(training_file, 'rb') as f:
#        deduper.readTraining(f)

In [17]:
dedupe.consoleLabel(deduper)

name : Aurore Raynaud
job : Céramiste
company : Fernandes S.A.R.L.
street_address : 120, avenue de Millet
city : Sainte ClaudeVille
state : GN
email : allardgerard@dbmail.com
user_name : oguillot

name : Aurore Raynaud
job : Cérste
company : Fernandes S.A.R.L.
street_address : 20, aveue de Miet
city : Saine CladeVille
state : GN
email : allarderard@dbmal.com
user_name : guill

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished
name : Luisa Reichmann
job : Desge, jewelley
company : Meyer Stift888 & Co. KG
street_address : Briemerallee 537
city : Konsan
state : Hamurg
email : takuenet@loeffle.com
user_name : iheite

name : Luisa Reichmann
job : Designer, jewellery
company : Meyer Stiftung & Co. KG
street_address : Briemerallee 537
city : Konstanz
state : Hamburg
email : ritakuehnert@loeffler.com
user_name : iheintze

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)rev

In [18]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 1.0
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, email), TfidfTextCanopyPredicate: (0.6, company))
INFO:dedupe.training:(LevenshteinCanopyPredicate: (4, job), SimplePredicate: (twoGramFingerprint, street_address))
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, street_address), TfidfNGramCanopyPredicate: (0.8, user_name))
INFO:dedupe.training:(SimplePredicate: (alphaNumericPredicate, email), TfidfNGramCanopyPredicate: (0.8, user_name))
INFO:dedupe.training:(SimplePredicate: (firstIntegerPredicate, street_address), TfidfTextCanopyPredicate: (0.8, email))
INFO:dedupe.training:(SimplePredicate: (alphaNumericPredicate, user_name), SimplePredicate: (sameFiveCharStartPredicate, company))
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, job), SimplePredicate: (wholeFieldPre

In [19]:
with open(training_file, 'w') as tf:
    deduper.writeTraining(tf)

In [20]:
dupes = deduper.match(customers.T.to_dict())

INFO:dedupe.canopy_index:Removing stop word com
INFO:dedupe.canopy_index:Removing stop word co
INFO:dedupe.canopy_index:Removing stop word om


In [33]:
dupes[:1]

[((14, 135), (1.0, 1.0))]

In [24]:
customers.iloc[[84,1600]]

,name,job,company,street_address,city,state,email,user_name
84,Dr. Catherine Sutton,"Engineer, maintenance",Ross LLC,13689 Morales Centers,North Sarah,New Mexico,lewisnicole@yahoo.com,clittle
1600,Dr. Catherine Sutton,Enginee maintenance,Ross LLC,13689 Morales Centers,North Sarah,New Mexico,ewinicoe@yaoo.com,little


In [25]:
customers.iloc[[156,484]]

,name,job,company,street_address,city,state,email,user_name
156,Joseph Nelson,"Psychotherapist, child",Johnson Ltd,9475 Mccoy Street Apt. 776,West Bonnie,Massachusetts,walterlance@johnson.org,ryanclarke
484,Joseph Nelson,Pychthrapit child,Joh5so5 Lt5,9475 Mccy Strt Apt 776,Wst Bonni,Mchuett,wltrln@johnsonorg,ryaclarke


### Exercise: Flag duplicates by adding 2 extra columns, one for confidence score and one for duplicate_ids

In [116]:
# %load ../solutions/dedupe.py
import numpy as np

dupe_dict = {}

for dupepair, confidence in dupes:
    dupe_dict[dupepair[0]] = {'pair': dupepair, 'confidence': confidence[0]}
    dupe_dict[dupepair[1]] = {'pair': dupepair, 'confidence': confidence[0]}

customers['duplicate_pair'] = customers.index.map(lambda i: dupe_dict[i].get('pair')
                                                  if i in dupe_dict else np.nan)
customers['confidence'] = customers.index.map(lambda i: dupe_dict[i].get('confidence')
                                              if i in dupe_dict else np.nan)


In [172]:
my_dict= {entry[0]:(dup[0],entry[1]) for dup in dupes for entry in zip(*dup)}
customers.head()

,name,job,company,street_address,city,state,email,user_name,duplicate_pair,confidence,duplicate_pair_2,confidence_2
0,Patricia Schaefer,"Programmer, systems",Estrada-Best,398 Paul Drive,Christianview,Delaware,lambdavid@gmail.com,ndavidson,NaN,NaN,NaN,NaN
1,Olivie Dubois,Ingénieur recherche et développement en agroal...,Moreno,rue Lucas Benard,Saint Anastasie-les-Bains,AR,berthelotjacqueline@mahe.fr,manonallain,NaN,NaN,NaN,NaN
2,Mary Davies-Kirk,Public affairs consultant,Baker Ltd,Flat 3\nPugh mews,Stanleyfurt,ZA,middletonconor@hotmail.com,colemanmichael,NaN,NaN,NaN,NaN
3,Miroslawa Eckbauer,Dispensing optician,Ladeck GmbH,Mijo-Lübs-Straße 12,Neubrandenburg,Berlin,sophia01@yahoo.de,romanjunitz,NaN,NaN,NaN,NaN
4,Richard Bauer,"Accountant, chartered certified",Hoffman-Rocha,6541 Rodriguez Wall,Carlosmouth,Texas,tross@jensen-ware.org,adam78,NaN,NaN,NaN,NaN


In [174]:
customers['duplicate_pair_2'] = customers.index.map(lambda idx:my_dict[idx][0] if idx in my_dict else np.nan)
customers['confidence_2'] = customers.index.map(lambda idx: my_dict[idx][1] if idx in my_dict else np.nan)

customers.iloc[14]

name                  Patricia Perrin-Boulay
job                            Géothermicien
company                          Didier SARL
street_address         92, rue Jérôme Arnaud
city                                  Dupont
state                                     ZW
email               augustebertin@bonnet.com
user_name                    cordierclaudine
duplicate_pair                     (14, 135)
confidence                                 1
duplicate_pair_2                   (14, 135)
confidence_2                               1
Name: 14, dtype: object

In [32]:
customers[customers.duplicate_pair.notna()][['duplicate_pair','confidence']].head()

,duplicate_pair,confidence
14,"(14, 135)",1.0
31,"(31, 1587)",1.0
84,"(84, 1600)",1.0
131,"(131, 1652)",1.0
135,"(14, 135)",1.0


In [175]:
customers[customers.confidence.notnull() == True].head()

,name,job,company,street_address,city,state,email,user_name,duplicate_pair,confidence,duplicate_pair_2,confidence_2
14,Patricia Perrin-Boulay,Géothermicien,Didier SARL,"92, rue Jérôme Arnaud",Dupont,ZW,augustebertin@bonnet.com,cordierclaudine,"(14, 135)",1.0,"(14, 135)",1.0
31,Manuela Pacheco Acevedo,Freight forwarder,Oller Inc,Glorieta de Gabriel Haro 10,Castellón,Álava,caromagdalena@yahoo.com,rivasamparo,"(31, 1587)",1.0,"(31, 1587)",1.0
84,Dr. Catherine Sutton,"Engineer, maintenance",Ross LLC,13689 Morales Centers,North Sarah,New Mexico,lewisnicole@yahoo.com,clittle,"(84, 1600)",1.0,"(84, 1600)",1.0
131,Jörg Henschel,Charity officer,Schinke AG,Henny-Lorch-Ring 484,Hohenstein-Ernstthal,Baden-Württemberg,hubertthies@hotmail.de,anne-katrin51,"(131, 1652)",1.0,"(131, 1652)",1.0
135,Patricia Perrin-Boulay,Géotemicien,Didier SARL,"92, rue Jérôme Araud",Dupont,ZW,augustebertin@bnnetcm,coclun,"(14, 135)",1.0,"(14, 135)",1.0
